In [41]:
import numpy as np
import pandas as pd

# ---- CÁC HÀM CỐT LÕI (DÙNG CHUNG) ----

def nhandathuc(b, c):
    """Nhân đa thức b(x) với (x - c). Quy ước: hệ số bậc cao -> thấp."""
    n = len(b)
    a = np.zeros(n + 1)
    a[0] = b[0]
    for i in range(1, n):
        a[i] = b[i] - b[i-1] * c
    a[n] = -b[n-1] * c
    return a

def hoocneNhan(z):
    """Tạo đa thức W(x) = (x-z₀)(x-z₁)..."""
    if len(z) == 0: return np.array([1.0])
    W = np.array([1.0])
    for val in z:
        W = nhandathuc(W, val)
    return W

def Tinh(W, c):
    """Tính giá trị đa thức W tại điểm c bằng Horner."""
    n = len(W) - 1
    if n < 0: return 0
    b = W[0]
    for i in range(1, n + 1):
        b = b * c + W[i]
    return b
    
def horner_steps(W, c):
    """Thực hiện Horner và trả về một list các bước trung gian."""
    n = len(W) - 1
    if n < 0: return []
    steps = []
    b = W[0]
    steps.append(b)
    for i in range(1, n + 1):
        b = b * c + W[i]
        steps.append(b)
    return steps

def format_poly_final(coeffs, precision=5):
    """In đa thức từ quy ước [a_n, ..., a_0]."""
    coeffs = np.round(coeffs, precision)
    poly_str = ""
    n = len(coeffs) - 1
    for i in range(len(coeffs)):
        coeff = coeffs[i]
        power = n - i
        if abs(coeff) < 1e-12: continue
        sign = " - " if coeff < 0 else " + "
        if poly_str == "": sign = "-" if sign == " - " else ""
        coeff = abs(coeff)
        if power == 0: term = f"{coeff}"
        elif power == 1: term = f"{coeff}*x"
        else: term = f"{coeff}*x^{power}"
        poly_str += f"{sign}{term}"
    return poly_str.strip()

# ---- HÀM TẠO BÀI GIẢI TỔNG HỢP CHO CÂU B ----

def baiGiaiLagrangeTheoMau(x_data, y_data):
    """
    Hàm này tạo ra bài giải tổng hợp chi tiết cho câu b) theo đúng mẫu bạn yêu cầu,
    bao gồm cả các dòng mô tả chi tiết.
    """

    # --- Bước 1: Chuẩn bị dữ liệu và tính toán ---
    x, y = x_data, y_data
    n = len(x)
    
    cy_list = []
    W_matrix = []
    for i in range(n):
        z = np.delete(x, i)
        W = hoocneNhan(z)
        W_matrix.append(W)
        c = Tinh(W, x[i])
        cy_list.append(y[i] / c)
        
    final_coeffs = np.sum([cy * W for cy, W in zip(cy_list, W_matrix)], axis=0)
    
    # --- Bước 2: Chuẩn bị dữ liệu cho các bảng để in ---
    
    # Bảng Tích
    tich_table_data = []
    W_tich = np.array([1.0])
    max_cols = n + 1
    for i in range(n):
        W_tich = nhandathuc(W_tich, x[i])
        padding = [0.0] * (max_cols - len(W_tich))
        row = [x[i]] + padding + list(W_tich)
        tich_table_data.append(row)
    tich_df = pd.DataFrame(tich_table_data)
    
    # Bảng Thương
    thuong_header = hoocneNhan(x) # Chính là W_{n+1}(x)
    thuong_body_data = []
    for i in range(n):
        steps = horner_steps(W_matrix[i], x[i])
        thuong_body_data.append(steps)
        
    full_thuong_table_data = [thuong_header] + thuong_body_data + [final_coeffs]
    
    table_index = [f'W_{n+1}(x)'] + [f"x_{i}={v}" for i, v in enumerate(x)] + ['P(x)']
    table_columns = [f"x^{n-j}" for j in range(n + 1)]
    thuong_df = pd.DataFrame(full_thuong_table_data, index=table_index, columns=table_columns)

    # --- Bước 3: In toàn bộ báo cáo theo mẫu ---
    print("\n   ================== BÀI GIẢI TỔNG HỢP (cho câu b) ==================\n")
    
    # 1. In Vector Cy
    print("   ciyi = yi / (xi-x0)...(xi-x(i-1))(xi-x(i+1))...(xi-xn)")
    print("   1. Vector Cy:")
    print("   Cy = [c0y0 c1y1 ... ciyi]")
    # Thiết lập định dạng in cho numpy để không bị tràn dòng
    np.set_printoptions(precision=8, suppress=True, linewidth=120)
    print("  ", np.array(cy_list))
    
    # 2. Lập Bảng Tích
    print("\n   2. Lập Bảng Tích:")
    # Cần thụt lề cho bảng để đẹp hơn
    indented_table = "   " + tich_df.to_string(header=False, index=False, float_format="{:,.4f}".format).replace("\n", "\n   ")
    print(indented_table)
    print(f"   Wn+1(x) = {format_poly_final(thuong_header, precision=4)}")
    
    # 3. Lập Bảng Thương
    print("\n   3. Lập bảng thương (chứa Ma trận A):")
    # Thay thế cột cuối cùng của các hàng trung gian bằng 0.0000
    for i in range(1, n + 1):
        thuong_df.iloc[i, -1] = 0.0
    indented_table_thuong = "   " + thuong_df.to_string(float_format="{:,.4f}".format).replace("\n", "\n   ")
    print(indented_table_thuong)
    
    # 4. Hệ số của đa thức cuối cùng P = Cy * A
    print("\n   4. Hệ số của đa thức cuối cùng P = Cy * A:")
    print("  ", final_coeffs)
    
    # 5. Kết luận P(x)
    print("\n   5. Kết luận P(x):")
    print(f"      P(x) = {format_poly_final(final_coeffs, precision=5)}")
    
    return final_coeffs

# ---- CHƯƠNG TRÌNH CHÍNH GIẢI BÀI TOÁN B3 ----

if __name__ == "__main__":
    x_full = np.array([2.1, 2.2, 2.4, 2.5, 2.7])
    y_full = np.array([3.178, 3.452, 3.597, 4.132, 4.376])

    print("="*60)
    print("Bài toán B13: Nội suy Lagrange")
    print("="*60)
    pd.options.display.float_format = '{:,.5f}'.format

    # --- Câu a) Xây dựng đa thức Lagrange cơ bản với mốc x_i = 2.5 (TRÌNH BÀY CHI TIẾT) ---
    print("\na) Xây dựng đa thức Lagrange cơ bản L_i(x) với mốc x_i = 2.5")
    
    xi = 2.5
    i_index = np.where(x_full == xi)[0][0]
    xj_nodes = np.delete(x_full, i_index)
    
    print(f"\n   Đa thức cơ bản L_{i_index}(x) có dạng: L_{i_index}(x) = Numerator(x) / Denominator")
    
    # -- 1. Xây dựng TỬ SỐ (Numerator) và Bảng Tích --
    print("\n   1. Xây dựng Tử số Numerator(x) = Π(x - x_j) với j != i:")
    
    tich_steps_data = []
    W_tich = np.array([1.0])
    max_poly_len = len(xj_nodes) + 1
    
    for node in xj_nodes:
        W_tich = nhandathuc(W_tich, node)
        padding = [0.0] * (max_poly_len - len(W_tich))
        row = [node] + padding + list(W_tich)
        tich_steps_data.append(row)
        
    tich_df = pd.DataFrame(tich_steps_data)
    num_powers = len(xj_nodes)
    tich_df.columns = ["Mốc nhân"] + [f"x^{num_powers-j}" for j in range(num_powers+1)]
    
    print("      Bảng Tích cho Tử số:")
    print(tich_df.to_string(index=False))
    
    numerator_poly = W_tich
    print(f"\n      => Tử số là: Numerator(x) = {format_poly_final(numerator_poly)}")

    # -- 2. Tính MẪU SỐ (Denominator) --
    print("\n   2. Tính Mẫu số Denominator = Π(x_i - x_j) với j != i:")
    
    factors_str = []
    denominator_value = 1.0
    for node in xj_nodes:
        term = xi - node
        factors_str.append(f"({xi} - {node})")
        denominator_value *= term
        
    print(f"      Phép tính: {' * '.join(factors_str)}")
    print(f"      => Mẫu số = {denominator_value:.5f}")

    # -- 3. Kết luận đa thức L_i(x) --
    Li_coeffs = numerator_poly / denominator_value
    
    print("\n   3. Kết luận đa thức L_i(x) = Numerator(x) / Denominator:")
    print(f"      L_{i_index}(x) = {format_poly_final(Li_coeffs)}")

    # --- Câu b) Xây dựng đa thức nội suy với 5 điểm đầu tiên (KHÔI PHỤC LẠI BÀI GIẢI TỔNG HỢP) ---
    print("\n" + "-"*60)
    print("\nb) Xây dựng đa thức nội suy Lagrange P(x) với 5 điểm dữ liệu đầu tiên")
    
    x_b = np.array([2.1, 2.2, 2.4, 2.5, 2.7])
    y_b = np.array([3.178, 3.452, 3.597, 4.132, 4.376])
    
    print("   Sử dụng các điểm:")
    print("   x =", x_b)
    print("   y =", y_b)
    
    # Gọi hàm để in bài giải chi tiết cho câu b
    P_b_coeffs = baiGiaiLagrangeTheoMau(x_b, y_b)
    
    # --- Câu c) Sử dụng kết quả câu b để tính f(2.8) và đánh giá sai số ---
    print("\n" + "-"*60)
    if P_b_coeffs is not None:
        print("\nc) Sử dụng P(x) từ câu b để tính f(2.8) và đánh giá sai số")
        x_eval = 2.8
        y_approx = Tinh(P_b_coeffs, x_eval)
        y_true_c = 4.954 # Giá trị thực của f(2.8) từ bảng gốc
        error = abs(y_true_c - y_approx)
        
        print(f"   Giá trị gần đúng tính được: P({x_eval}) = {y_approx:.5f}")
        print(f"   Giá trị thực trong bảng:    f({x_eval}) = {y_true_c:.5f}")
        print(f"   Sai số tuyệt đối = |{y_true_c} - {y_approx:.5f}| = {error:.5f}")
    
    print("\n" + "="*60)

Bài toán B13: Nội suy Lagrange

a) Xây dựng đa thức Lagrange cơ bản L_i(x) với mốc x_i = 2.5

   Đa thức cơ bản L_3(x) có dạng: L_3(x) = Numerator(x) / Denominator

   1. Xây dựng Tử số Numerator(x) = Π(x - x_j) với j != i:
      Bảng Tích cho Tử số:
 Mốc nhân     x^4      x^3      x^2       x^1       x^0
  2.10000 0.00000  0.00000  0.00000   1.00000  -2.10000
  2.20000 0.00000  0.00000  1.00000  -4.30000   4.62000
  2.40000 0.00000  1.00000 -6.70000  14.94000 -11.08800
  2.70000 1.00000 -9.40000 33.03000 -51.42600  29.93760

      => Tử số là: Numerator(x) = 1.0*x^4 - 9.4*x^3 + 33.03*x^2 - 51.426*x + 29.9376

   2. Tính Mẫu số Denominator = Π(x_i - x_j) với j != i:
      Phép tính: (2.5 - 2.1) * (2.5 - 2.2) * (2.5 - 2.4) * (2.5 - 2.7)
      => Mẫu số = -0.00240

   3. Kết luận đa thức L_i(x) = Numerator(x) / Denominator:
      L_3(x) = -416.66667*x^4 + 3916.66667*x^3 - 13762.5*x^2 + 21427.5*x - 12474.0

------------------------------------------------------------

b) Xây dựng đa thức 